In [15]:
# NER test
import pandas as pd
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)




INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [16]:
df_tail = pd.read_csv('S:\Workspace\MORETECH\nlp-workbench\lenta-news-sample-1000.csv', index_col=0)
text = df_tail['text'].iloc[0]
print(text)

Генеральный директор Первого канала Константин Эрнст рассказал о том, как осуществляется контроль за контентом на российском телевидении. Комментарий медиаменеджера опубликован в журнале New Yorker. «Никто никогда не говорит вам: "Не показывайте Навального (Алексей Навальный, блогер — прим. «Ленты.ру»), не используйте его имя". Такие поручения не передаются словами», — так Эрнст ответил на вопрос о том, есть ли цензура на телевидении. Он добавил, что федеральными каналами руководят неглупые люди. Телеведущая Юлия Панкратова, работавшая на Первом канале с 2006 по 2013 год, подтвердила, что в большинстве случаев сотрудникам не поступало прямых указаний от руководства по поводу контента. В беседе с журналистом New Yorker она призналась, что от нее чаще ожидали интуитивного понимания правил. Константин Эрнст возглавляет Первый канал с 1999 года. Он начал карьеру на телевидении в 1988 году в программе «Взгляд», затем стал автором и ведущим программы «Матадор». С 6 октября 1999 года журналис

In [17]:
# Usage example

navec = Navec.load(r'S:\Workspace\MORETECH\nlp-workbench\models\navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load(r'S:\Workspace\MORETECH\nlp-workbench\models\slovnet_ner_news_v1.tar')
_ = ner.navec(navec)

markup = ner(text)
show_markup(markup.text, markup.spans)

Генеральный директор Первого канала Константин Эрнст рассказал о том, 
                     ORG─────────── PER─────────────                  
как осуществляется контроль за контентом на российском телевидении. 
Комментарий медиаменеджера опубликован в журнале New Yorker. «Никто 
                                                 ORG───────         
никогда не говорит вам: "Не показывайте Навального (Алексей Навальный,
                                        PER───────  PER────────────── 
 блогер — прим. «Ленты.ру»), не используйте его имя". Такие поручения 
                 ORG─────                                             
не передаются словами», — так Эрнст ответил на вопрос о том, есть ли 
                              PER──                                  
цензура на телевидении. Он добавил, что федеральными каналами 
руководят неглупые люди. Телеведущая Юлия Панкратова, работавшая на 
                                     PER────────────                
Первом канале с 2006 по 20

In [18]:
# Example for production

navec = Navec.load(r'S:\Workspace\MORETECH\nlp-workbench\models\navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load(r'S:\Workspace\MORETECH\nlp-workbench\models\slovnet_ner_news_v1.tar')
_ = ner.navec(navec)

def get_ners_dict(text: str, ner_model) -> dict:
    markup = ner_model(text)
    ORGS, PERS, LOCS = [], [], []
    for span in markup.spans:
        span_text = markup.text[span.start:span.stop]
        if span.type == 'ORG':
            ORGS.append(span_text)
        elif span.type == 'PER':
            PERS.append(span_text)
        elif span.type == 'LOC':
            LOCS.append(span_text)

        # print(span.type, span_text)

    return {'ORGs': ORGS, 'PERs': PERS, 'LOCs': LOCS}



from pprint import pprint
ner_dict = get_ners_dict(text, ner)
pprint(ner_dict)



{'LOCs': [],
 'ORGs': ['Первого канала',
          'New Yorker',
          'Ленты.ру',
          'Первом канале',
          'New Yorker',
          'Первый канал',
          'Взгляд',
          'Матадор',
          'ОРТ',
          'Первый канал'],
 'PERs': ['Константин Эрнст',
          'Навального',
          'Алексей Навальный',
          'Эрнст',
          'Юлия Панкратова',
          'Константин Эрнст',
          'Максим Галкин']}


In [19]:
# Get all entities from text
def get_entities_func(ner):
    return lambda text: ner(text).spans
df_tail['entities'] = df_tail['text'].parallel_apply(get_entities_func(ner))
df_tail.head()

,title,text,topic,tags,date,source,entities
url,,,,,,,
https://lenta.ru/news/2019/12/10/ernst/,Эрнст рассказал о цензуре на российском телеви...,Генеральный директор Первого канала Константин...,NaN,ТВ и радио,2019/12/10,lenta,"[Span(start=21, stop=35, type='ORG'), Span(sta..."
https://lenta.ru/news/2019/12/10/saudi/,Крупнейшая сделка в истории поменяла цель,Власти Саудовской Аравии с помощью первичного ...,NaN,Рынки,2019/12/10,lenta,"[Span(start=7, stop=24, type='LOC'), Span(star..."
https://lenta.ru/news/2019/12/10/ponomarev/,Путин рассказал о тайной связи правозащитника ...,Лидер ликвидированного правозащитного движения...,NaN,Политика,2019/12/10,lenta,"[Span(start=48, stop=65, type='ORG'), Span(sta..."
https://lenta.ru/news/2019/12/10/hailsatan/,Стюардессы оскорбились внешним видом пассажирк...,Пассажирка авиакомпании American Airlines пожа...,NaN,Происшествия,2019/12/10,lenta,"[Span(start=24, stop=41, type='ORG'), Span(sta..."
https://lenta.ru/news/2019/12/10/zenit/,«Зенит» крупно проиграл «Бенфике» и вылетел из...,Петербургский «Зенит» крупно проиграл лиссабон...,NaN,Футбол,2019/12/10,lenta,"[Span(start=15, stop=20, type='ORG'), Span(sta..."


In [20]:
df_tail.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, https://lenta.ru/news/2019/12/10/ernst/ to https://lenta.ru/news/2019/12/14/olimp/
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     1000 non-null   object 
 1   text      1000 non-null   object 
 2   topic     0 non-null      float64
 3   tags      993 non-null    object 
 4   date      1000 non-null   object 
 5   source    1000 non-null   object 
 6   entities  1000 non-null   object 
dtypes: float64(1), object(6)
memory usage: 62.5+ KB
